In [18]:

import os
import json
import time
import base64
import requests

import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1
        
        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df


def get_data(symbol, bar):
    
    d = requests.get(f'https://www.okx.com/api/v5/market/index-candles?instId={symbol}&bar={bar}').json()
    data = pd.DataFrame(d['data'],columns=['timestamp','open','high','low','close','confirm'])
    data['timestamp'] = pd.to_datetime(data['timestamp'],utc=True, unit='ms')
    for i in ['open','high','low','close']:
        data[i] = data[i].astype(float)
    data = data.set_index('timestamp').sort_index()
    return data

def get_current_trend(symbol, bar):
    data = get_data(symbol, bar)
    data = supertrend(data)
    
    return bool(data.iloc[len(data)-1]['in_uptrend'])

In [19]:

from aevo import AevoClient
from clientConfig import CLIENT_CONFIG

from datetime import datetime
from loguru import logger as log



# Add Credentials
aevo = AevoClient(**CLIENT_CONFIG)


In [33]:


c = 0
def run_bot(symbol,symbol_id,is_trading_up,amount,ticks):
    
    
    global c
    c+=1
    aevo.rest_cancel_all_orders()

    print(f"[CLEAR] clear existing positions {datetime.now().isoformat()}")
    while len(aevo.rest_get_data("positions")['positions'])>0:
        pos = aevo.rest_get_data("positions")['positions'][0]
        side = pos['side']
        pos_amount = float(pos['amount'])

        if side == 'buy':
            aevo.rest_create_market_order(symbol_id,False,pos_amount)
        else:
            aevo.rest_create_market_order(symbol_id,True,pos_amount)
        time.sleep(5)
        aevo.rest_cancel_all_orders()

    
    print(f"[OREDER] epoch {c} time {datetime.now().isoformat()}")

    direction = 'asks' if is_trading_up else 'bids'
    aevo.rest_create_market_order(symbol_id,is_trading_up,amount)
    aevo.rest_create_order(symbol_id,not is_trading_up, float(aevo.get_orderbook(symbol)[direction][ticks][0]),amount) 
    open_time = datetime.now()

    print(f"opentime {open_time.isoformat()} checking all position filled")

    while len(aevo.rest_get_data("positions")['positions'])>0 or len(aevo.rest_get_open_orders()) > 0 :
        print(f"{datetime.now().isoformat()} position not filled")
        if (datetime.now() - open_time).seconds > 60*20:
            print(f"[TIMEOUT] timeout {datetime.now().isoformat()}")
            break
        time.sleep(10)

    

while True:
    run_bot('ETH-PERP',1,get_current_trend('ETH-USDT', '15m'),0.2,10)
    time.sleep(1)

[CLEAR] clear existing positions 2024-02-22T15:03:28.250472


2024-02-22 15:03:28.556 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}


[OREDER] epoch 1 time 2024-02-22T15:03:28.555413


2024-02-22 15:03:29.284 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-22 15:03:29.292 | INFO     | aevo:rest_create_order:196 - {'maker': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'is_buy': False, 'instrument': 1, 'limit_price': '2963950000', 'amount': '200000', 'salt': '8458624631', 'signature': '0x161e587c89ad1d13ab9e1abb882fd9ea81fbb3b6a30ae35bce832e8f2fc4cb740271023f16ef69f2a37abb15b70fd303f1d32a3f3e26050c2ebeb04d4c6ba1271c', 'post_only': True, 'reduce_only': False, 'close_position': False, 'timestamp': 1708585409}


opentime 2024-02-22T15:03:29.611581 checking all position filled
2024-02-22T15:03:29.921414 position not filled
2024-02-22T15:03:40.237777 position not filled
2024-02-22T15:03:50.588021 position not filled
[CLEAR] clear existing positions 2024-02-22T15:04:02.839448


...

2024-02-22 15:19:45.009 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}


[OREDER] epoch 14 time 2024-02-22T15:19:45.009975


2024-02-22 15:19:45.725 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-22 15:19:45.733 | INFO     | aevo:rest_create_order:196 - {'maker': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'is_buy': False, 'instrument': 1, 'limit_price': '2977300000', 'amount': '200000', 'salt': '2403796161', 'signature': '0xc2f07b51b98226eacfbf0a919f430d2e5d0af43bb8af972e07994f50873c4130584dcd30c31dbbfedba3ef309e25378544d976d8071cabd19858727f7103a0841b', 'post_only': True, 'reduce_only': False, 'close_position': False, 'timestamp': 1708586385}


opentime 2024-02-22T15:19:46.050676 checking all position filled
2024-02-22T15:19:46.373676 position not filled
2024-02-22T15:19:56.687770 position not filled
2024-02-22T15:20:07.048850 position not filled
2024-02-22T15:20:17.357340 position not filled
2024-02-22T15:20:27.687326 position not filled
2024-02-22T15:20:38.025614 position not filled
2024-02-22T15:20:48.350149 position not filled
2024-02-22T15:20:58.670361 position not filled
2024-02-22T15:21:08.991738 position not filled
2024-02-22T15:21:19.342869 position not filled
2024-02-22T15:21:29.659451 position not filled
2024-02-22T15:21:39.977253 position not filled
2024-02-22T15:21:50.293140 position not filled
2024-02-22T15:22:00.613570 position not filled
2024-02-22T15:22:10.929876 position not filled
2024-02-22T15:22:21.273637 position not filled
2024-02-22T15:22:31.587264 position not filled
2024-02-22T15:22:41.928720 position not filled
2024-02-22T15:22:52.244178 position not filled
2024-02-22T15:23:02.561151 position not fi

KeyboardInterrupt: 

In [ ]:
import schedule
def clear_all_positions(symbol_id):
    print(f"[CLEAR] clear existing positions {datetime.now().isoformat()}")
    aevo.rest_cancel_all_orders()
    while len(aevo.rest_get_data("positions")['positions'])>0:
        pos = aevo.rest_get_data("positions")['positions'][0]
        side = pos['side']
        pos_amount = float(pos['amount'])

        if side == 'buy':
            aevo.rest_create_market_order(symbol_id,False,pos_amount)
        else:
            aevo.rest_create_market_order(symbol_id,True,pos_amount)
        time.sleep(5)
        aevo.rest_cancel_all_orders()

def check_buy_sell_signals(df):
    global in_position

    print("checking for buy and sell signals")
    # print(df.tail(5))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1

    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("changed to uptrend, buy")
        if not in_position:
            clear_all_positions(5197)
            print("[BUY ORDER]")
            aevo.rest_create_market_order(5197,True,10)
            
            in_position = True
        else:
            print("already in position, nothing to do")
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position:
            print("changed to downtrend, sell")
            clear_all_positions(5197)
            print("[SELL ORDER]\n")
            aevo.rest_create_market_order(5197,False,10)
            in_position = False
        else:
            print("You aren't in position, nothing to sell")

def run_bot():
    print(f"Fetching new bars for {datetime.now().isoformat()}")
    data = get_data('SOL-USDT', '4h')
    supertrend_data = supertrend(data)
    check_buy_sell_signals(supertrend_data)


schedule.every(10).seconds.do(run_bot)


while True:
    schedule.run_pending()
    time.sleep(1)


In [14]:
get_data('SOL-USDT', '4H')

,open,high,low,close,confirm
timestamp,,,,,
2024-02-05 12:00:18.944000+00:00,97.88,98.69,95.24,95.60,1
2024-02-05 16:00:36.864000+00:00,95.60,96.52,95.35,95.94,1
2024-02-05 20:00:54.784000+00:00,95.94,96.11,94.24,95.65,1
2024-02-05 23:59:01.632000+00:00,95.65,96.79,95.21,96.01,1
2024-02-06 03:59:19.552000+00:00,96.01,96.10,94.97,95.76,1
2024-02-06 07:59:37.472000+00:00,95.76,96.33,93.16,94.32,1
2024-02-06 11:59:55.392000+00:00,94.32,95.85,93.34,95.75,1
2024-02-06 16:00:13.312000+00:00,95.75,97.66,95.14,97.57,1
2024-02-06 20:00:31.232000+00:00,97.58,97.88,96.75,96.86,1


In [17]:
get_data('SOL-USDT', '15m')

,open,high,low,close,confirm
timestamp,,,,,
2024-02-21 01:00:51.584000+00:00,107.39,108.06,107.15,107.96,1
2024-02-21 01:13:58.016000+00:00,107.96,108.24,107.65,107.70,1
2024-02-21 01:29:15.520000+00:00,107.69,107.91,107.26,107.33,1
2024-02-21 01:44:33.024000+00:00,107.34,107.41,106.99,107.09,1
2024-02-21 01:59:50.528000+00:00,107.11,107.32,106.65,106.85,1
2024-02-21 02:15:08.032000+00:00,106.85,107.05,106.41,106.66,1
2024-02-21 02:30:25.536000+00:00,106.66,107.11,106.54,107.00,1
2024-02-21 02:45:43.040000+00:00,107.00,107.09,106.66,106.78,1
2024-02-21 03:01:00.544000+00:00,106.79,107.06,106.74,106.89,1


In [ ]:
aevo.rest